In [2]:
pip install selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB 187.9 kB/s eta 0:00:50
   ---------------------------------------- 0.0/9.4 MB 187.9 kB/s eta 0:00:50
   ---------------------------------------- 0.0/9.4 MB 164.3 kB/s eta 0:00:57
   ---------------------------------------- 0.0/9.4 MB 164.3 kB/s eta 0:00:57
   ---------------------------------------- 0.1/9.4 MB 193.2 kB/s eta 0:00:49
   ---------------------------------------- 0.1/9.4 M


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\sarum\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\sarum\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager  
import pandas as pd
import time

def scrape_ipl_commentary_selenium(match_url):
    service = Service(ChromeDriverManager().install())  
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--window-size=1920,1080')
    
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        driver.get(match_url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".cb-com-ln"))
        )
        
        try:
            match_id = match_url.split('/')[-2] 
            series_name = driver.find_element(By.CSS_SELECTOR, ".cb-nav-subhdr a").text
            match_no = driver.find_element(By.CSS_SELECTOR, ".cb-nav-subhdr span").text.split(',')[0]
            match_type = "IPL"
            toss_info = driver.find_element(By.CSS_SELECTOR, ".cb-text-toss").text
            winning_team = driver.find_element(By.CSS_SELECTOR, ".cb-text-complete").text if driver.find_elements(By.CSS_SELECTOR, ".cb-text-complete") else "N/A"
            tie_breaker = "Super Over" if "Super Over" in winning_team else "-"
        except Exception as e:
            print(f"Error extracting match info: {e}")
            match_id = "N/A"
            series_name = "Indian Premier League"
            match_no = "1"
            match_type = "IPL"
            toss_info = "N/A"
            winning_team = "N/A"
            tie_breaker = "-"
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
        
        commentary_items = driver.find_elements(By.CSS_SELECTOR, ".cb-com-ln")
        data = []
        
        for item in commentary_items:
            try:
                over_ball = item.find_element(By.CSS_SELECTOR, ".cb-com-ln-num").text.strip()
                commentary_text = item.find_element(By.CSS_SELECTOR, ".cb-com-ln-comm").text.strip()
                over_no, ball_in_over = over_ball.split('.')
                
                data.append({
                    'year': 2021,
                    'series_type': 'T20',
                    'series_name': series_name,
                    'match_no': match_no,
                    'match_type': match_type,
                    'match_id': match_id,
                    'match_winning_team': winning_team,
                    'match_tie_breaker': tie_breaker,
                    'match_toss': toss_info,
                    'ball_no': f"{over_no}.{ball_in_over}",  
                    'over_no': over_no,
                    'ball_commentary': commentary_text
                })
            except Exception as e:
                print(f"Error processing commentary item: {e}")
                continue
        
        return pd.DataFrame(data)
        
    except Exception as e:
        print(f"Error during scraping: {e}")
        return None
    finally:
        driver.quit()
match_url = "https://www.cricbuzz.com/live-cricket-commentary/12345/rcb-vs-mi-1st-match-ipl-2021"
commentary_df = scrape_ipl_commentary_selenium(match_url)

if commentary_df is not None and not commentary_df.empty:
    print(commentary_df.head())
    commentary_df.to_csv('rcb_vs_mi_ipl2021_ball_by_ball.csv', index=False)
else:
    print("Failed to scrape commentary or no data found.")

Error extracting match info: Message: no such element: Unable to locate element: {"method":"css selector","selector":".cb-text-toss"}
  (Session info: chrome=135.0.7049.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00697FD3+60707]
	GetHandleVerifier [0x00698014+60772]
	(No symbol) [0x004C0683]
	(No symbol) [0x00508660]
	(No symbol) [0x005089FB]
	(No symbol) [0x00551022]
	(No symbol) [0x0052D094]
	(No symbol) [0x0054E824]
	(No symbol) [0x0052CE46]
	(No symbol) [0x004FC5D3]
	(No symbol) [0x004FD424]
	GetHandleVerifier [0x008DBAB3+2435075]
	GetHandleVerifier [0x008D7053+2416035]
	GetHandleVerifier [0x008F33FC+2531660]
	GetHandleVerifier [0x006AF0A5+155125]
	GetHandleVerifier [0x006B5A4D+182173]
	GetHandleVerifier [0x0069F8A8+91640]
	GetHandleVerifier [0x0069FA50+92064]
	GetHandleVerifier [0x0068A510+4704]
	BaseThreadInitThunk [0x75835D49+25]
	RtlInitial